In [43]:
# Flag to force to reload dataset
RELOAD = False

In [44]:
import os

# import Kaggle API to load dataset
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

# initialize Kaggle API
api = KaggleApi()
api.authenticate()

# download dataset from Kaggle to data folder
data_path = 'data'
api.competition_download_files('foursquare-location-matching', data_path, force=RELOAD, quiet=False)
# save filename: !ATTENTION! : it may not be wroking if many files are in folders
# then just name it manually 
dataset_file_name = os.listdir(data_path)[0]

foursquare-location-matching.zip: Skipping, found more recently modified local copy (use --force to force download)


In [45]:
# import libraries to work with paths and to read zipped file, as was downloaded from Kaggle
from zipfile import ZipFile
# import pandas for EDA
import pandas as pd

# Read train dataset (train.csv) to pandas DataFrame named df: it will be used for analysis
df = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('train.csv'))


In [46]:
# Check, that all dataframes are loaded and have correct shapes
print(f'Shape of df: {str(df.shape)}')

Shape of df: (1138812, 13)


In [58]:
# Sample 10 000 records, it should give 100 000 000 records
# Sample 1 000 records, it should give 1 000 000 records
# 100 samples = 25 seconds
# 500 samples = 100 minutes
# numbers above are for iteration
# Bellow for merging
# 100 samples = < 1 second
# 500 samples = < 1 second
# 1000 samples = 1.5 seconds + 15 seconds to save
# 10 000 samples = 30 seconds product + MemoryError: Unable to allocate 17.9 GiB for an array with shape (24, 100000000) and data type object after 5 minutes
# 5000 samples = 10 seconds product + 1m45s merging + 6 minutes to save (7 Gb file)
df_sample = df.sample(5000, random_state=42)

In [59]:
from itertools import product

combo_df = pd.DataFrame(data=list(product(df_sample['id'], df_sample['id'])), columns=['id_1','id_2'])
#list(product(df_sample['id'], df_sample['id']))

In [60]:
combo_df.shape

(25000000, 2)

In [61]:
merged_df = combo_df.merge(df_sample, how='inner', left_on='id_1', right_on='id')
df_pairs_custom = merged_df.merge(df_sample, how='inner', left_on='id_2', right_on='id')
df_pairs_custom.drop(['id_1', 'id_2'], axis=1, inplace=True)
df_pairs_custom['match'] = df_pairs_custom['point_of_interest_x'] == df_pairs_custom['point_of_interest_y']
df_pairs_custom.drop(['point_of_interest_x', 'point_of_interest_y'], axis=1, inplace=True)
df_pairs_custom.columns=['id_1', 'name_1', 'latitude_1', 'longitude_1', 'address_1', 'city_1', 'state_1', 'zip_1', 'country_1', 'url_1', 'phone_1', 'categories_1',
                                'id_2', 'name_2', 'latitude_2', 'longitude_2', 'address_2', 'city_2', 'state_2', 'zip_2', 'country_2', 'url_2', 'phone_2', 'categories_2', 'match']

In [62]:
# for index, row in combo_df.iterrows():
#         # id_1 = row['id_1']
#         # id_2 = row['id_2']

#         rec1 = df_sample[df_sample['id'] == row['id_1']]
#         rec2 = df_sample[df_sample['id'] == row['id_2']]

#         #print(rec1, rec2)

#         #print(rec1['name'])

#         dict_1 = {'id_1' : row['id_1'], 'name_1' : rec1['name'].values, 'latitude_1' : rec1['latitude'].values, 'longitude_1' : rec1['longitude'].values, 'address_1' : rec1['address'].values, 'city_1' : rec1['city'].values, 
#                 'state_1' : rec1['state'].values, 'zip_1' : rec1['zip'].values, 'country_1' : rec1['country'].values, 'url_1' : rec1['url'].values, 'phone_1' : rec1['phone'].values, 'categories_1' : rec1['categories'].values,
#                 'id_2' : row['id_2'], 'name_2' : rec2['name'].values, 'latitude_2' : rec2['latitude'].values, 'longitude_2' : rec2['longitude'].values, 'address_2' : rec2['address'].values, 'city_2' : rec2['city'].values, 
#                 'state_2' : rec2['state'], 'zip_2' : rec2['zip'], 'country_2' : rec2['country'], 'url_2' : rec2['url'], 'phone_2' : rec2['phone'], 'categories_2' : rec2['categories'],
#                 'match' : (rec1['point_of_interest'].values == rec2['point_of_interest'].values)}

#         #print(dict_1)

#         # new_record = pd.DataFrame({'id_1' : row['id_1'], 'name_1' : rec1['name'], 'latitude_1' : rec1['latitude'], 'longitude_1' : rec1['longitude'], 'address_1' : rec1['address'], 'city_1' : rec1['city'], 
#         #         'state_1' : rec1['state'], 'zip_1' : rec1['zip'], 'country_1' : rec1['country'], 'url_1' : rec1['url'], 'phone_1' : rec1['phone'], 'categories_1' : rec1['categories'],
#         #         'id_2' : row['id_2'], 'name_2' : rec2['name'], 'latitude_2' : rec2['latitude'], 'longitude_2' : rec2['longitude'], 'address_2' : rec2['address'], 'city_2' : rec2['city'], 
#         #         'state_2' : rec2['state'], 'zip_2' : rec2['zip'], 'country_2' : rec2['country'], 'url_2' : rec2['url'], 'phone_2' : rec2['phone'], 'categories_2' : rec2['categories'],
#         #         'match' : (rec1['point_of_interest'].values == rec2['point_of_interest'].values)}, index=[0])

#         new_record = pd.DataFrame(dict_1)
#         #print(new_record)

#         df_pairs_custom = pd.concat([df_pairs_custom, new_record], ignore_index=True)


In [63]:
df_pairs_custom

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,phone_1,categories_1,id_2,name_2,latitude_2,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,True
1,E_f96f973047a099,Rüstem Paşa (Ulu) Camii,40.359018,30.012008,Camikebir Mh. Sakarya Cd.,Osmaneli,Bilecik,NaN,TR,NaN,NaN,"Mosques, Historic Sites",E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
2,E_b16dafc319e90a,Starbucks,27.870459,-82.761560,10809 Starkey Rd,Largo,FL,33777,US,http://www.starbucks.com/store/1018298,(727) 392-3435,Coffee Shops,E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
3,E_0be920c4d49631,Refter Sint Tarcisiusinstituut,50.832716,5.102211,NaN,NaN,NaN,NaN,BE,NaN,NaN,"High Schools, Cafeterias",E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
4,E_10d09c32c73bb3,Wetherspoon Express,51.890014,0.262341,"Satellite 2, near Gate 31",Stansted,Essex,CM24 1QW,GB,http://www.jdwetherspoon.co.uk,1279669040,Pubs,E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24999995,E_ebe957a8eadccc,กาแฟนอกใจ (พี่จิ๊บ),13.844291,100.632547,ซ.รามอินทรา 34 แยก 21 (อยู่เย็น),Tha Raeng,Bangkok,10230,TH,NaN,NaN,"Coffee Shops, Cafés",E_c1d2e18ba2e744,Wedangan Kawatan,-7.575253,110.815760,Kawatan Street,NaN,NaN,NaN,ID,NaN,NaN,Coffee Shops,False
24999996,E_193fc3ad23ec66,Парк гойдалок,50.380755,30.478977,NaN,Київ,Київ,NaN,UA,NaN,NaN,Playgrounds,E_c1d2e18ba2e744,Wedangan Kawatan,-7.575253,110.815760,Kawatan Street,NaN,NaN,NaN,ID,NaN,NaN,Coffee Shops,False
24999997,E_7cf35decf90a41,Anthem Preparatory Academy,33.844925,-112.137633,39808 N Gavilan Peak Pkwy,Anthem,AZ,85086,US,http://www.anthemprep.org,6234654776,Schools,E_c1d2e18ba2e744,Wedangan Kawatan,-7.575253,110.815760,Kawatan Street,NaN,NaN,NaN,ID,NaN,NaN,Coffee Shops,False
24999998,E_fb9a847796772e,Cengelkoy Sıtesı Otopark,41.048777,29.054854,NaN,Istanbul,NaN,NaN,TR,NaN,NaN,NaN,E_c1d2e18ba2e744,Wedangan Kawatan,-7.575253,110.815760,Kawatan Street,NaN,NaN,NaN,ID,NaN,NaN,Coffee Shops,False


In [64]:
# for index1, rec1 in df_sample.iterrows():
#     for index2, rec2 in df_sample.iterrows():
#         if ((rec1['id'] != rec2['id'])):
#             new_record = pd.DataFrame({'id_1' : rec1['id'], 'name_1' : rec1['name'], 'latitude_1' : rec1['latitude'], 'longitude_1' : rec1['longitude'], 'address_1' : rec1['address'], 'city_1' : rec1['city'], 
#             'state_1' : rec1['state'], 'zip_1' : rec1['zip'], 'country_1' : rec1['country'], 'url_1' : rec1['url'], 'phone_1' : rec1['phone'], 'categories_1' : rec1['categories'],
#             'id_2' : rec2['id'], 'name_2' : rec2['name'], 'latitude_2' : rec2['latitude'], 'longitude_2' : rec2['longitude'], 'address_2' : rec2['address'], 'city_2' : rec2['city'], 
#             'state_2' : rec2['state'], 'zip_2' : rec2['zip'], 'country_2' : rec2['country'], 'url_2' : rec2['url'], 'phone_2' : rec2['phone'], 'categories_2' : rec2['categories'],
#             'match' : (rec1['point_of_interest'] == rec2['point_of_interest'])}, index=[0])

#             df_pairs_custom = pd.concat([df_pairs_custom, new_record], ignore_index=True)
#             # df_pairs_custom.append({'id_1' : rec1['id'], 'name_1' : rec1['name'], 'latitude_1' : rec1['latitude'], 'longitude_1' : rec1['longitude'], 'address_1' : rec1['address'], 'city_1' : rec1['city'], 
#             # 'state_1' : rec1['state'], 'zip_1' : rec1['zip'], 'country_1' : rec1['country'], 'url_1' : rec1['url'], 'phone_1' : rec1['phone'], 'categories_1' : rec1['categories'],
#             # 'id_2' : rec2['id'], 'name_2' : rec2['name'], 'latitude_2' : rec2['latitude'], 'longitude_2' : rec2['longitude'], 'address_2' : rec2['address'], 'city_2' : rec2['city'], 
#             # 'state_2' : rec2['state'], 'zip_2' : rec2['zip'], 'country_2' : rec2['country'], 'url_2' : rec2['url'], 'phone_2' : rec2['phone'], 'categories_2' : rec2['categories']}, ignore_index=True)


In [65]:
# save to csv after ~60 minutes only 144092 records are done (out of 1000000)
df_pairs_custom.to_csv('.\\data\\pairs_custom.csv', index=False)

In [66]:
pd.set_option('display.max_columns', None)
df_pairs_custom.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,phone_1,categories_1,id_2,name_2,latitude_2,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_9c455c75918751,Manjabal 2,-6.712090,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,E_9c455c75918751,Manjabal 2,-6.71209,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,True
1,E_f96f973047a099,Rüstem Paşa (Ulu) Camii,40.359018,30.012008,Camikebir Mh. Sakarya Cd.,Osmaneli,Bilecik,NaN,TR,NaN,NaN,"Mosques, Historic Sites",E_9c455c75918751,Manjabal 2,-6.71209,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
2,E_b16dafc319e90a,Starbucks,27.870459,-82.761560,10809 Starkey Rd,Largo,FL,33777,US,http://www.starbucks.com/store/1018298,(727) 392-3435,Coffee Shops,E_9c455c75918751,Manjabal 2,-6.71209,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
3,E_0be920c4d49631,Refter Sint Tarcisiusinstituut,50.832716,5.102211,NaN,NaN,NaN,NaN,BE,NaN,NaN,"High Schools, Cafeterias",E_9c455c75918751,Manjabal 2,-6.71209,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False
4,E_10d09c32c73bb3,Wetherspoon Express,51.890014,0.262341,"Satellite 2, near Gate 31",Stansted,Essex,CM24 1QW,GB,http://www.jdwetherspoon.co.uk,1279669040,Pubs,E_9c455c75918751,Manjabal 2,-6.71209,108.561016,Jalan Karanggetas,Cirebon,Jawa Barat,NaN,ID,NaN,NaN,Indonesian Restaurants,False


In [67]:
df_pairs_custom.shape

(25000000, 25)